# word2vec
word2vecにはCBOWとskip-gramが存在。
精度がいいのはskip-gram、計算速度が速いのはCBOW。<br>  
CBOWはコンテキストから単語を推測、skip-gramは単語から周囲のコンテキストを推測。
高速化のために
- embeddingレイヤの実装
- Negative Sampling<br>
  
が存在

### CBOW
CBOWモデルはコンテキストからターゲットを推測するモデル。  
（前後1単語から推測する場合は）入力層が二つ、中間層が一つ、出力層で出来ている。  
出力層は各単語のスコアを計算し、最終的にsoftmax関数を用いてターゲットとなる単語を選ぶ。  
損失関数は
$$ L = -\frac{1}{T}\sum^T_{t=1}\log{P(w_t | w_{t-1}, w_{t+1})}$$


### skip-gram
CBOWモデルを逆転させたような構造。  
前後1単語を予測する場合は次のような損失関数
$$ L= -\frac{1}{T}\sum^T_{t=1}(\log{P(w_{t-1}|w_t)+log{P(w_{t+1}|w_t)}}) $$


参考url
- https://qiita.com/g-k/items/69afa87c73654af49d36


In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from collections import Counter
import random
import pandas as pd

df = pd.read_csv('dataset/token.csv')
df = df[1:40]
sentences = []
for text in df['article']:
    text_list = text.split(' ')
    sentences.append(text_list)

from gensim.models import Word2Vec
model = Word2Vec(sentences,  sg=1, vector_size=100, window=5, min_count=1)

In [12]:
for word, similarity in model.wv.most_similar('家族'):
    print(f"{word}: {similarity}")


行く: 0.9988073706626892
通勤: 0.9986675381660461
分: 0.9986549019813538
返信: 0.9986339807510376
なかなか: 0.998598039150238
支給: 0.9985608458518982
様子: 0.9985296130180359
位: 0.9985253214836121
初回: 0.9985184669494629
理解: 0.9985105991363525


In [17]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"現在使用中のデバイス: {device}")


現在使用中のデバイス: cpu


/home/st6322041/anaconda3/envs/ML/lib/python3.9/site-packages/torch/cuda/__init__.py:174: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11070). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at /pytorch/c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


In [14]:
results = model.wv.most_similar(positive=['人生'], negative=['幸福'])
for word, similarity in results:
    print(f"{word}: {similarity:.4f}")


KeyError: "Key '幸福' not present in vocabulary"